In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


# Setting up Project
Set inputs and outputs

In [2]:
# Import helpful libraries
import pandas as pd
from xgboost import XGBRegressor
#from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Load the data, and separate the target
iowa_file_path = "/kaggle/input/home-data-for-ml-course/train.csv"
home_data = pd.read_csv(iowa_file_path)

In [3]:
y = home_data.SalePrice

# Select columns corresponding to features, and preview the data
X = home_data
X.drop(['SalePrice'], axis=1, inplace=True)
X.head()

# Split into validation and training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1)

# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and 
                    X_train[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

In [4]:
X

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


# Cleaning Up the Data
Use a pipeline to make code simpler and easier to understand

Understanding null values

In [5]:
# get the number of missing data points per column
missing_values_count = X.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Id                0
MSSubClass        0
MSZoning          0
LotFrontage     259
LotArea           0
Street            0
Alley          1369
LotShape          0
LandContour       0
Utilities         0
dtype: int64

In [6]:
total_cells = np.product(X.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

5.9631849315068495


In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [8]:
xgb_model = XGBRegressor(random_state = 1)
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', xgb_model)
                             ])
print("completed")

completed


In [9]:
"""
import scipy.sparse
columns = X.columns
#pd.DataFrame.sparse.from_spmatrix(preprocessor.fit_transform(X),)
pd.DataFrame(preprocessor.fit_transform(X), columns = columns)
"""

'\nimport scipy.sparse\ncolumns = X.columns\n#pd.DataFrame.sparse.from_spmatrix(preprocessor.fit_transform(X),)\npd.DataFrame(preprocessor.fit_transform(X), columns = columns)\n'

# Hyperparameter Tuning

In [10]:
from sklearn.model_selection import GridSearchCV
"""
params = { 'model__max_depth': [3,5, 6],
           'model__learning_rate': [0.05],
           'model__n_estimators': [750, 100]}

grid_search = GridSearchCV(estimator=pipeline, 
                   param_grid=params,
                   scoring='neg_root_mean_squared_error', 
                   verbose=1)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Lowest RMSE: ", (-grid_search.best_score_))

"""

'\nparams = { \'model__max_depth\': [3,5, 6],\n           \'model__learning_rate\': [0.05],\n           \'model__n_estimators\': [750, 100]}\n\ngrid_search = GridSearchCV(estimator=pipeline, \n                   param_grid=params,\n                   scoring=\'neg_root_mean_squared_error\', \n                   verbose=1)\ngrid_search.fit(X_train, y_train)\n\nprint("Best parameters:", grid_search.best_params_)\nprint("Lowest RMSE: ", (-grid_search.best_score_))\n\n'

# Creating Final ML Model
This will be used to check the MAE score on all the training data and will also be used for the final submission

In [11]:
# Define a XGBoost model
xgb_best_model = XGBRegressor(random_state=1, learning_rate = 0.05, max_depth = 3, n_estimators = 750,)

print("completed")

completed


In [12]:
# Train model on half data to see MAE
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', xgb_best_model)
                             ])

# Preprocessing of training data, fit model 

my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 14971.796746575343


In [13]:
# Preprocessing of training data, fit model 
my_pipeline.fit(X, y)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                                                   'GrLivArea', 'BsmtFullBath',
                                                   'BsmtHalfBath', 'F...
                              gamma=0, gpu_id=-1

# Submitting the code

In [14]:
# path to file you will use for predictions
test_data_path = '/kaggle/input/home-data-for-ml-course/test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)


# make predictions which we will submit. 
test_preds = my_pipeline.predict(test_data)

In [15]:
# Run the code to save predictions in the format used for competition scoring

output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)